# Diffusion Coefficient Calculation in Liquid Argon

This notebook simulates a three-dimensional system of Argon atoms interacting via the Lennard-Jones (LJ) potential to compute the mean squared displacement (MSD) and extract diffusion coefficients under different thermostat conditions.

---

## Configuration Parameters (via `Configuration` class):
- `dimensions`: Simulation dimensionality (fixed to 3D).
- `n_particles`: Number of Argon atoms (default: 1000).
- `density`: Mass density of Argon ($1374 \, \mathrm{kg/m^3}$).
- `dt`: Timestep used in dynamics ($2 \times 10^{-15} \, \mathrm{s}$).
- `steps`: Total number of simulation steps (default: 2000).
- `temperature`: Initial and target temperature ($94.4 \, \mathrm{K}$).
- `sigma`, `epsilon`: Lennard-Jones parameters for Argon ($\sigma = 3.4 \times 10^{-10}\,\mathrm{m}$, $\epsilon = 1.656 \times 10^{-21}\,\mathrm{J}$).
- `rcutoff`: Cutoff distance for LJ interactions ($2.5\sigma$).
- `tau_ber`: Relaxation time for Berendsen thermostat.
- `gamma_langevin`: Friction constant for Langevin thermostat.
- `thermostat_type`: Thermostat method (`"nve"`, `"berendsen"`, or `"langevin"`).

---

## Key Functions and Return Values:
- `compute_lj_force(r, sigma, epsilon, rcutoff)`: Computes the Lennard-Jones force between two particles.
- `compute_lj_potential(r, sigma, epsilon, rcutoff)`: Computes the shifted Lennard-Jones potential energy.
- `build_linked_cells(positions, box_size, rcutoff)`: Constructs linked-cell neighbor lists for efficient force calculation.
- `compute_forces_lca(...)`: Computes interatomic forces using the linked-cell algorithm.
- `apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau)`: Applies Berendsen thermostat to regulate system temperature.
- `apply_langevin_thermostat(velocities, gamma, dt, target_temp)`: Applies Langevin thermostat with stochastic and friction forces.
- `create_lattice(n_particles, box_size, dimensions)`: Initializes particle positions on a slightly perturbed cubic lattice.
- `initialize_velocities(n_particles, dimensions, target_temp)`: Initializes particle velocities from a Maxwell-Boltzmann distribution.
- `run_simulation_with_msd(config)`: Runs molecular dynamics simulation, returns temperature evolution and MSD over time.

---

## Simulation:
- Initializes a system of 1000 Argon atoms within a periodic cubic box.
- Calculates initial forces via the Lennard-Jones potential truncated at $r_{\mathrm{cutoff}}$.
- Integrates the equations of motion using the velocity-Verlet algorithm.
- Applies either no thermostat (NVE), Berendsen thermostat, or Langevin thermostat.
- Tracks the mean squared displacement (MSD) as a function of time.
- Extracts diffusion coefficients from the slope of MSD vs time via Einstein's relation: $D = \frac{1}{2d} \frac{d}{dt} \mathrm{MSD}(t)$
- Visualizes and compares the MSD behavior under different thermostatting methods.

---

## Parameters:
| Parameter         | Value                      |
|-------------------|-----------------------------|
| dimensions        | 3                           |
| n_particles       | 1000                        |
| density           | 1374 $\mathrm{kg/m^3}$       |
| dt                | $2 \times 10^{-15}\,\mathrm{s}$ |
| steps             | 2000                        |
| temperature       | 94.4 $\mathrm{K}$            |
| sigma             | $3.4 \times 10^{-10}\,\mathrm{m}$ |
| epsilon           | $1.656 \times 10^{-21}\,\mathrm{J}$ |
| rcutoff           | $2.5\sigma$                  |
| tau_ber           | $1 \times 10^{-13}\,\mathrm{s}$ |
| gamma_langevin    | $1 \times 10^{13}\,\mathrm{s^{-1}}$ |



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

#Parameters:
sigma = 3.4e-10  # meters
epsilon = 1.656e-21  # Joules
mass_argon = 39.95 * 1.66054e-27  # kg
density_kg_m3 = 1374  # kg/m³

def compute_lj_force(r, sigma, epsilon, rcutoff):
    if r >= rcutoff or r < 1e-12:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    return 24 * epsilon * (2 * inv_r12 - inv_r6) / r

def compute_lj_potential(r, sigma, epsilon, rcutoff):
    if r >= rcutoff:
        return 0.0
    inv_r = sigma / r
    inv_r6 = inv_r ** 6
    inv_r12 = inv_r6 ** 2
    pot = 4 * epsilon * (inv_r12 - inv_r6)
    inv_rcut = sigma / rcutoff
    shift = 4 * epsilon * (inv_rcut ** 12 - inv_rcut ** 6)
    return pot - shift

def build_linked_cells(positions, box_size, rcutoff):
    n_particles, dim = positions.shape
    lc = max(1, int(np.floor(box_size / rcutoff)))
    lc_dim = [lc] * dim
    rc = box_size / lc
    EMPTY = -1
    head = [EMPTY] * (lc ** dim)
    lscl = [EMPTY] * n_particles

    for i in range(n_particles):
        mc = [int(positions[i][a] / rc) for a in range(dim)]
        mc = [min(max(0, idx), lc - 1) for idx in mc]
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        lscl[i] = head[c_index]
        head[c_index] = i

    return head, lscl, lc_dim

def compute_forces_lca(positions, box_size, rcutoff, sigma, epsilon, use_pbc=True):
    n_particles, dim = positions.shape
    head, lscl, lc_dim = build_linked_cells(positions, box_size, rcutoff)
    EMPTY = -1
    forces = np.zeros_like(positions)
    neighbor_offsets = np.array(np.meshgrid(*[[-1, 0, 1]] * dim)).T.reshape(-1, dim)

    for mc in np.ndindex(*lc_dim):
        if dim == 2:
            c_index = mc[0] * lc_dim[1] + mc[1]
        else:
            c_index = mc[0] * lc_dim[1] * lc_dim[2] + mc[1] * lc_dim[2] + mc[2]
        i = head[c_index]
        while i != EMPTY:
            pos_i = positions[i]
            for offset in neighbor_offsets:
                mc1 = np.array(mc) + offset
                rshift = np.zeros(dim)
                valid_cell = True
                for a in range(dim):
                    if use_pbc:
                        if mc1[a] < 0:
                            mc1[a] += lc_dim[a]
                            rshift[a] = -box_size
                        elif mc1[a] >= lc_dim[a]:
                            mc1[a] -= lc_dim[a]
                            rshift[a] = box_size
                    else:
                        if mc1[a] < 0 or mc1[a] >= lc_dim[a]:
                            valid_cell = False
                            break
                if not valid_cell:
                    continue
                if dim == 2:
                    c1 = mc1[0] * lc_dim[1] + mc1[1]
                else:
                    c1 = mc1[0] * lc_dim[1] * lc_dim[2] + mc1[1] * lc_dim[2] + mc1[2]
                j = head[c1]
                while j != EMPTY:
                    if j > i:
                        pos_j = positions[j] + rshift
                        r_ij = pos_i - pos_j
                        dist = np.linalg.norm(r_ij)
                        if dist < rcutoff and dist > 1e-12:
                            f_mag = compute_lj_force(dist, sigma, epsilon, rcutoff)
                            fij = f_mag * (r_ij / dist)
                            forces[i] += fij
                            forces[j] -= fij
                    j = lscl[j]
            i = lscl[i]
    return forces, 0.0

#Configurations:
class Configuration:
    def __init__(self):
        self.dimensions = 3
        self.n_particles = 1000
        self.density = density_kg_m3
        self.dt = 2e-15
        self.steps = 2000
        self.temperature = 94.4
        self.sigma = sigma
        self.epsilon = epsilon
        self.rcutoff = 2.5 * sigma
        self.tau_ber = 1e-13
        self.gamma_langevin = 1e13
        self.thermostat_type = 'langevin'


def create_lattice(n_particles, box_size, dimensions):
    n_side = int(np.ceil(n_particles ** (1 / dimensions)))
    spacing = box_size / n_side
    positions = []
    for indices in np.ndindex(*([n_side] * dimensions)):
        if len(positions) < n_particles:
            pos = [(i + 0.5) * spacing for i in indices]
            noise = np.random.uniform(-0.05, 0.05, size=dimensions) * spacing
            positions.append(np.array(pos) + noise)
    return np.array(positions)

def initialize_velocities(n_particles, dimensions, target_temp):
    kb = 1.380649e-23  # J/K
    velocities = np.random.normal(0, 1, size=(n_particles, dimensions))
    velocities -= np.mean(velocities, axis=0)
    ke = 0.5 * mass_argon * np.sum(velocities**2)
    dof = n_particles * dimensions
    scale = np.sqrt(target_temp * kb * dof / (2 * ke))
    velocities *= scale
    return velocities


def apply_berendsen_thermostat(velocities, target_temp, current_temp, dt, tau):
    lambda_factor = np.sqrt(1 + dt / tau * (target_temp / current_temp - 1))
    return velocities * lambda_factor

def apply_langevin_thermostat(velocities, gamma, dt, target_temp):
    kb = 1.380649e-23  # J/K
    noise = np.random.normal(0, 1, size=velocities.shape)
    c1 = np.exp(-gamma * dt)
    c2 = np.sqrt((1 - c1**2) * kb * target_temp / mass_argon)
    return c1 * velocities + c2 * noise

def run_simulation_with_msd(config):
    box_size = (config.n_particles * mass_argon / config.density) ** (1 / config.dimensions)
    positions = create_lattice(config.n_particles, box_size, config.dimensions)
    velocities = initialize_velocities(config.n_particles, config.dimensions, config.temperature)
    initial_positions = positions.copy()
    dof = config.n_particles * config.dimensions

    temp_list = []
    msd_list = []

    for step in tqdm(range(config.steps), desc=f"Running {config.thermostat_type}"):
        forces, _ = compute_forces_lca(positions, box_size, config.rcutoff,
                                       config.sigma, config.epsilon, use_pbc=True)
        velocities += 0.5 * forces / mass_argon * config.dt
        positions += velocities * config.dt
        positions %= box_size
        forces, _ = compute_forces_lca(positions, box_size, config.rcutoff,
                                       config.sigma, config.epsilon, use_pbc=True)
        velocities += 0.5 * forces / mass_argon * config.dt

        kinetic = 0.5 * mass_argon * np.sum(velocities ** 2)
        temp = 2 * kinetic / (dof * 1.380649e-23)
        temp_list.append(temp)

        if config.thermostat_type == 'berendsen':
            velocities = apply_berendsen_thermostat(velocities, config.temperature, temp, config.dt, config.tau_ber)
        elif config.thermostat_type == 'langevin':
            velocities = apply_langevin_thermostat(velocities, config.gamma_langevin, config.dt, config.temperature)

        displacement = positions - initial_positions
        displacement -= box_size * np.round(displacement / box_size)
        squared_displacement = np.sum(displacement ** 2, axis=1)
        msd = np.mean(squared_displacement)
        msd_list.append(msd)

    return temp_list, msd_list


names = ['nve', 'berendsen', 'langevin']
thermostats = [None, 'berendsen', 'langevin']
colors = ['gray', 'red', 'blue']

results_temp = {}
results_msd = {}
diffusion_constants = {}

for name, method in zip(names, thermostats):
    cfg = Configuration()
    cfg.thermostat_type = method if method else 'none'
    cfg.steps = 2000
    temps, msds = run_simulation_with_msd(cfg)
    results_temp[name] = temps
    results_msd[name] = msds
    time_array = np.arange(len(msds)) * cfg.dt
    slope, intercept = np.polyfit(time_array, msds, 1)
    D = slope / (2 * cfg.dimensions) 
    diffusion_constants[name] = D

    print(f"{name.upper()} done: Final T = {np.mean(temps[-100:]):.2f} K, Final MSD = {msds[-1]:.4e} m², D = {D:.4e} m²/s")

#PLot:
# plt.figure(figsize=(8, 5))
# for name, color in zip(names, colors):
#     time_array = np.arange(len(results_msd[name])) * Configuration().dt
#     plt.plot(time_array, results_msd[name], label=name.capitalize(), color=color)
# plt.xlabel('Time (s)')
# plt.ylabel('MSD (m²)')
# plt.title('MSD vs Time (Argon)')
# plt.legend()
# plt.grid()
# plt.tight_layout()
# plt.show()